# Data Cleaning Appendix

### By Erik Williams, Christine Chau, and Danny Rusk

The following are the questions that we wish to find the answers to with the help of our project and analyses: 
1) Is there a relationship between the levels of carcinogens in drinking water and leukemia incidence rates?
2) Is there a relationship between childhood blood-lead levels and leukemia incidence rate?
3) Do incidence rate trends correlate to carcinogen levels?

Through our research combing of the web, our team decided to use data from multiple sources including The Pennsylvania Department of Health — which yielded our arsenic and HAA5 data, The National Cancer Institute — which yielded our data regarding incidence rates of leukemia, and finally The Center for Disease Control — which yielded data regarding blood lead levels in the Pennsylvania. 

Because of the variety in sourcing of our data, alongside the variability within the different data collected among all of these studies, we conducted extensive data cleaning/ pre-processing, and labeling in order to obtain analysis-ready data. 

The following were the steps that we took in conducting this data cleaning: 

1. First, we combined data on blood lead levels with data on leukemia incidence rates by county. In order to accomplish this task, we had to delete various extraneous columsn which were prroduced when we obtained the datasets form the CDC and NCI. An example of such a column is 'County Fips' in the blood-lead level data, which was not necessary for our analysis. 

2. We also had to adjust the formatting of the blood-lead data. It was initially formatted to have an overarching header column over blood levels, titled 'Number of Children with Confirmed BLLs by BLL Group'. We removed this column and renamed every column under it to correctly describe what the column described.
  
3. Next, we merged the data. This was carried out by first converting each dataframe's respective 'County' column to the same format. We ran into some issues because of inconsistencies within the naming of counties. The counties within the cancer dataframe had a trailing space, so when we tried to merge, pandas could not find any counties that were in both the blood and cancer dataframes. By altering the county names of each county within the  blood dataframe, we got matching county names. We then identified which counties were in both dataframes.

4. Additionally, we converted all percentages, which were strings, into numbers. This allows for easier data analysis
  
5. We then identified which counties were in both dataframes.

6. Finally, we merged the two dataframes by county name with the merge function. We then parameterized the process of cleaning the data and made it into a method so we could reuse it with data sets from different years. For this phase, we dropped all counties which had n/a for either percentage of children with confirmed blood lead levels above 5 or 10 micrograms
  
7. Next, we combined this newly created dataframe with th data regarding arsenic and HAA5 levels in each county. This process was much easier. The main action we had to take was to average the data for each county. For example, a single county could have 10 water treatment plants. We averaged this data and weighted it by population served t ensure an accurate analysis. 
  
8. After this, we had to drop a single column which was named 'Unnamed: 0' from both the arsenic and HAA5 datasets.

9. Additionally, we needed to convert all of the numbers in these new datasets to floats as they were originally strings. 
  
10. We merged the datasets by county
11. We removed all columns not relevant to our data analysis. 

In the following cells you can see the processes we have described above demonstrated in our code. The final output, represented by the "pa" dataframe (outputted at the bottom) is the fully cleaned, merged, and final dataframe that was used in our analyses for our project.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
from scipy.stats import poisson, ttest_ind

def readble(csv1,csv2,year):
    """"
    param csv1: a string, path to blood data file
    param csv2: a string, path to cancer data file
    param year: an int, the year the blood data is from
    return: merged dataframe
    """
    
#import blood data
    pa_blood = pd.read_csv(csv1, skiprows= 2)
    if type(csv2)== str:
        pa_cancer = pd.read_csv(csv2)
    else:
        pa_cancer= csv2

#deleted blood columns that we do not need
    del pa_blood['County FIPS']
    del pa_blood['Number of Children with Confirmed BLLs by BLL Group']
    pa_blood = pa_blood.rename(columns={'Unnamed: 10' : 'Number 5-9 µg/dL'})

#renamed blood columns to be more comprehensible 
    pa_blood = pa_blood.rename(columns={'Children with Confirmed BLLs ≥ 5 µg/dL': str(year) + " Number of Children with Confirmed BLLs ≥ 5 µg/dL"})
    pa_blood = pa_blood.rename(columns={'Unnamed: 6': str(year) + " Percentage of Children with Confirmed BLLs ≥ 5 µg/dL"})
    pa_blood = pa_blood.rename(columns={'Children with Confirmed BLLs ≥ 10 µg/dL': str(year) + " Number of Children with Confirmed BLLs ≥ 10 µg/dL"})
    pa_blood = pa_blood.rename(columns={'Unnamed: 8': str(year) + " Percentage of Children with Confirmed BLLs ≥ 10 µg/dL"})
    pa_blood = pa_blood.rename(columns={'Unnamed: 10' : str(year) + ' Number 5-9 µg/dL'})
    pa_blood = pa_blood.rename(columns={'Unnamed: 11' : str(year) + ' Number 10-14 µg/dL'})
    pa_blood = pa_blood.rename(columns={'Unnamed: 12' : str(year) + ' Number 15-19-9 µg/dL'})
    pa_blood = pa_blood.rename(columns={'Unnamed: 13' : str(year) + ' Number 20-24 µg/dL'})
    pa_blood = pa_blood.rename(columns={'Unnamed: 14' : str(year) + ' Number 25-44 µg/dL'})
    pa_blood = pa_blood.rename(columns={'Unnamed: 15' : str(year) + ' Number ≥ 45 µg/dL'})
    pa_blood = pa_blood.rename(columns={'Total Population of Children < 72 Months of Age' : str(year)+' Total Population of Children < 72 Months of Age'})
    pa_blood = pa_blood.rename(columns={'Number of Children Tested < 72 Months of Age' : str(year)+' Number of Children Tested < 72 Months of Age'})
    pa_blood = pa_blood.rename(columns={'Percentage of Children Tested < 72 Months of Age' : str(year)+' Percentage of Children Tested < 72 Months of Age'})

#delete bad cancer df column
    if type(csv2)== str:
        pa_cancer = pa_cancer.drop(pa_cancer.index[[1]])

#create df of PA counties in the cancer df
    pa_c_counties = pa_cancer['County']

    good_counties = []
    fixedcounty = []

#adjust county names in blood df to include a trailing space to match county names in cancer df
    if type(csv2)== str:
        for county in pa_cancer['County']:
            x = len(county)
            if county.find('6')!= -1:
                fixedcounty.append(str(county)[:x-3])
            else:
                fixedcounty.append(str(county)[:x-1])
    if type(csv2)== str:
        pa_cancer['County'] = fixedcounty
    else:
        fixedcounty = pa_cancer['County'].values.tolist()
#convert county name df to list in order to iterate over it
    pa_c_counties = pa_c_counties.values.tolist()

#find each county that is in both the blood and cancer df
    for county in fixedcounty:
        if county in pa_c_counties:
            good_counties.append(county)
    pa_cancer.head()
    pa_cancer = pa_cancer[pa_cancer['County'].isin(good_counties)]
    pa_blood = pa_blood[pa_blood['County Name'].isin(good_counties)]
    if type(csv2) == str:
        pa_cancer = pa_cancer.drop(pa_cancer.index[[0]])

    pa_blood = pa_blood.rename(columns = {'County Name' : 'County'})
    pa = pd.merge(pa_blood,pa_cancer,on='County')

    return pa

#Call readable three times to merge the total of 4 datasets
pa = readble('pa_blood.csv','pa_cancer.csv',2016)
pa = readble('pa_blood2015.csv',pa,2015)
pa = readble('pa_blood2014.csv',pa,2014)
pa.to_csv(r'Users\erikwilliams\Info2950Project\pa.csv', index = False)

paCopy = pa
pa = pa.dropna(axis = 'rows',subset=['2016 Percentage of Children with Confirmed BLLs ≥ 5 µg/dL', '2015 Percentage of Children with Confirmed BLLs ≥ 5 µg/dL', '2014 Percentage of Children with Confirmed BLLs ≥ 5 µg/dL'])
pa = pa.dropna(axis = 'rows',subset=['2016 Percentage of Children with Confirmed BLLs ≥ 10 µg/dL', '2015 Percentage of Children with Confirmed BLLs ≥ 10 µg/dL', '2014 Percentage of Children with Confirmed BLLs ≥ 10 µg/dL'])
pa = pa.drop(pa.index[pa['Age-Adjusted Incidence Rate(‚Ä†) - cases per 100,000'] == '* '])

In [3]:
arsenic = pd.read_csv('WaterData.csv')
arsenic = arsenic.drop(columns ='Unnamed: 0')
pa = pd.merge(pa,arsenic, on='County')
arsenic = pd.read_csv('WaterData2.csv')
arsenic.head()
arsenic = arsenic.drop(columns ='Unnamed: 0')
pa = pd.merge(pa,arsenic, on='County')
pa.head()

,County,2014 Total Population of Children < 72 Months of Age,2014 Number of Children Tested < 72 Months of Age,2014 Percentage of Children Tested < 72 Months of Age,2014 Number of Children with Confirmed BLLs ≥ 5 µg/dL,2014 Percentage of Children with Confirmed BLLs ≥ 5 µg/dL,2014 Number of Children with Confirmed BLLs ≥ 10 µg/dL,2014 Percentage of Children with Confirmed BLLs ≥ 10 µg/dL,Number 5-9 µg/dL,2014 Number 10-14 µg/dL,...,Recent Trend,Recent 5-Year Trend (‚Ä°) in Incidence Rates,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1,Weighted_Average_2016_Arsenic,Weighted_Average_2015_Arsenic,Weighted_Average_2014_Arsenic,Weighted_Average_2016_HAA5,Weighted_Average_2015_HAA5,Weighted_Average_2014_HAA5
0,Adams County,"6,491",950,14.6%,78,8.2%,15,1.6%,11,NaN,...,stable,2.2,-0.1,4.4,0.700000,1.495946,0.7,6.104211,5.805758,9.026538
1,Allegheny County,"76,981","13,699",17.8%,733,5.4%,163,1.2%,100,29,...,stable,-1.9,-5,1.4,0.613636,0.641176,0.663636364,15.983889,18.495556,14.252286
2,Armstrong County,"4,151",839,20.2%,69,8.2%,14,1.7%,9,NaN,...,stable,-1.4,-4.2,1.5,0.650000,0.868750,0.88,15.458947,16.782778,10.829524
3,Beaver County,"10,721","1,601",14.9%,130,8.1%,31,1.9%,18,8,...,stable,1.2,-0.7,3.1,0.916667,1.044615,0.833333333,13.998519,14.681304,11.861600
4,Berks County,"29,837","4,079",13.7%,101,2.5%,41,1.0%,16,13,...,stable,0.7,-1,2.4,0.825455,1.154098,0.820833333,5.652097,6.944490,9.818605


In [4]:
#convert values contained within Arsenic and HAAS cols from str to float
pa['Weighted_Average_2014_Arsenic'] = pd.to_numeric(pa['Weighted_Average_2014_Arsenic'], errors = 'coerce')
pa = pa.replace(np.nan, 0, regex = True)
#type(pa['Weighted_Average_2014_Arsenic'][0])

pa['Weighted_Average_2015_Arsenic'] = pd.to_numeric(pa['Weighted_Average_2015_Arsenic'], errors = 'coerce')
pa = pa.replace(np.nan, 0, regex = True)
#type(pa['Weighted_Average_2015_Arsenic'][0])

pa['Weighted_Average_2016_Arsenic'] = pd.to_numeric(pa['Weighted_Average_2016_Arsenic'], errors = 'coerce')
pa = pa.replace(np.nan, 0, regex = True)
#type(pa['Weighted_Average_2016_Arsenic'][0])

pa['Weighted_Average_2014_HAA5'] = pd.to_numeric(pa['Weighted_Average_2014_HAA5'], errors = 'coerce')
pa = pa.replace(np.nan, 0, regex = True)
#type(pa['Weighted_Average_2014_HAAS'][0])

pa['Weighted_Average_2015_HAA5'] = pd.to_numeric(pa['Weighted_Average_2015_HAA5'], errors = 'coerce')
pa = pa.replace(np.nan, 0, regex = True)
#type(pa['Weighted_Average_2015_HAAS'][0])

pa['Weighted_Average_2016_HAA5'] = pd.to_numeric(pa['Weighted_Average_2016_HAA5'], errors = 'coerce')
pa = pa.replace(np.nan, 0, regex = True)
#type(pa['Weighted_Average_2016_HAAS'][0])


In [5]:
new = pa['2014 Percentage of Children Tested < 72 Months of Age']
new = new.tolist()
corrected = []
for x in new:
    y = x[:-1]
    corrected.append(y)
pa['2014 Percentage of Children Tested < 72 Months of Age'] = corrected

In [6]:
new = pa['2014 Percentage of Children with Confirmed BLLs ≥ 5 µg/dL']
new = new.tolist()
corrected = []
for x in new:
    y = x[:-1]
    corrected.append(y)
pa['2014 Percentage of Children with Confirmed BLLs ≥ 5 µg/dL'] = corrected

In [7]:
new = pa['2014 Percentage of Children with Confirmed BLLs ≥ 10 µg/dL']
new = new.tolist()
corrected = []
for x in new:
    y = x[:-1]
    corrected.append(y)
pa['2014 Percentage of Children with Confirmed BLLs ≥ 10 µg/dL'] = corrected

In [8]:
new = pa['2015 Percentage of Children Tested < 72 Months of Age']
new = new.tolist()
corrected = []
for x in new:
    y = x[:-1]
    corrected.append(y)
pa['2015 Percentage of Children Tested < 72 Months of Age'] = corrected

In [9]:
new = pa['2015 Percentage of Children with Confirmed BLLs ≥ 5 µg/dL']
new = new.tolist()
corrected = []
for x in new:
    y = x[:-1]
    corrected.append(y)
pa['2015 Percentage of Children with Confirmed BLLs ≥ 5 µg/dL'] = corrected



In [10]:
new = pa['2015 Percentage of Children with Confirmed BLLs ≥ 10 µg/dL']
new = new.tolist()
corrected = []
for x in new:
    y = x[:-1]
    corrected.append(y)
pa['2015 Percentage of Children with Confirmed BLLs ≥ 10 µg/dL'] = corrected

In [11]:
new = pa['2016 Percentage of Children Tested < 72 Months of Age']
new = new.tolist()
corrected = []
for x in new:
    y = x[:-1]
    corrected.append(y)
pa['2016 Percentage of Children Tested < 72 Months of Age'] = corrected

In [12]:
new = pa['2016 Percentage of Children with Confirmed BLLs ≥ 5 µg/dL']
new = new.tolist()
corrected = []
for x in new:
    y = x[:-1]
    corrected.append(y)
pa['2016 Percentage of Children with Confirmed BLLs ≥ 5 µg/dL'] = corrected

In [13]:
new = pa['2016 Percentage of Children with Confirmed BLLs ≥ 10 µg/dL']
new = new.tolist()
corrected = []
for x in new:
    y = x[:-1]
    corrected.append(y)
pa['2016 Percentage of Children with Confirmed BLLs ≥ 10 µg/dL'] = corrected

In [14]:
pa.head() 

,County,2014 Total Population of Children < 72 Months of Age,2014 Number of Children Tested < 72 Months of Age,2014 Percentage of Children Tested < 72 Months of Age,2014 Number of Children with Confirmed BLLs ≥ 5 µg/dL,2014 Percentage of Children with Confirmed BLLs ≥ 5 µg/dL,2014 Number of Children with Confirmed BLLs ≥ 10 µg/dL,2014 Percentage of Children with Confirmed BLLs ≥ 10 µg/dL,Number 5-9 µg/dL,2014 Number 10-14 µg/dL,...,Recent Trend,Recent 5-Year Trend (‚Ä°) in Incidence Rates,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1,Weighted_Average_2016_Arsenic,Weighted_Average_2015_Arsenic,Weighted_Average_2014_Arsenic,Weighted_Average_2016_HAA5,Weighted_Average_2015_HAA5,Weighted_Average_2014_HAA5
0,Adams County,"6,491",950,14.6,78,8.2,15,1.6,11,0,...,stable,2.2,-0.1,4.4,0.700000,1.495946,0.700000,6.104211,5.805758,9.026538
1,Allegheny County,"76,981","13,699",17.8,733,5.4,163,1.2,100,29,...,stable,-1.9,-5,1.4,0.613636,0.641176,0.663636,15.983889,18.495556,14.252286
2,Armstrong County,"4,151",839,20.2,69,8.2,14,1.7,9,0,...,stable,-1.4,-4.2,1.5,0.650000,0.868750,0.880000,15.458947,16.782778,10.829524
3,Beaver County,"10,721","1,601",14.9,130,8.1,31,1.9,18,8,...,stable,1.2,-0.7,3.1,0.916667,1.044615,0.833333,13.998519,14.681304,11.861600
4,Berks County,"29,837","4,079",13.7,101,2.5,41,1.0,16,13,...,stable,0.7,-1,2.4,0.825455,1.154098,0.820833,5.652097,6.944490,9.818605


In [15]:
#convert values contained within Arsenic and HAAS cols from str to float
pa['2014 Percentage of Children Tested < 72 Months of Age'] = pd.to_numeric(pa['2014 Percentage of Children Tested < 72 Months of Age'], errors = 'coerce')
pa = pa.replace(np.nan, 0, regex = True)
#type(pa['Weighted_Average_2014_Arsenic'][0])

pa['2014 Percentage of Children with Confirmed BLLs ≥ 5 µg/dL'] = pd.to_numeric(pa['2014 Percentage of Children with Confirmed BLLs ≥ 5 µg/dL'], errors = 'coerce')
pa = pa.replace(np.nan, 0, regex = True)
#type(pa['Weighted_Average_2015_Arsenic'][0])

pa['2014 Percentage of Children with Confirmed BLLs ≥ 10 µg/dL'] = pd.to_numeric(pa['2014 Percentage of Children with Confirmed BLLs ≥ 10 µg/dL'], errors = 'coerce')
pa = pa.replace(np.nan, 0, regex = True)
#type(pa['Weighted_Average_2016_Arsenic'][0])

pa['2015 Percentage of Children Tested < 72 Months of Age'] = pd.to_numeric(pa['2015 Percentage of Children Tested < 72 Months of Age'], errors = 'coerce')
pa = pa.replace(np.nan, 0, regex = True)
#type(pa['Weighted_Average_2014_HAAS'][0])

pa['2015 Percentage of Children with Confirmed BLLs ≥ 5 µg/dL'] = pd.to_numeric(pa['2015 Percentage of Children with Confirmed BLLs ≥ 5 µg/dL'], errors = 'coerce')
pa = pa.replace(np.nan, 0, regex = True)
#type(pa['Weighted_Average_2015_HAAS'][0])

pa['2015 Percentage of Children with Confirmed BLLs ≥ 10 µg/dL'] = pd.to_numeric(pa['2015 Percentage of Children with Confirmed BLLs ≥ 10 µg/dL'], errors = 'coerce')
pa = pa.replace(np.nan, 0, regex = True)
#type(pa['Weighted_Average_2016_HAAS'][0])
#convert values contained within Arsenic and HAAS cols from str to float
pa['2016 Percentage of Children Tested < 72 Months of Age'] = pd.to_numeric(pa['2016 Percentage of Children Tested < 72 Months of Age'], errors = 'coerce')
pa = pa.replace(np.nan, 0, regex = True)
#type(pa['Weighted_Average_2014_Arsenic'][0])

pa['2016 Percentage of Children with Confirmed BLLs ≥ 5 µg/dL'] = pd.to_numeric(pa['2016 Percentage of Children with Confirmed BLLs ≥ 5 µg/dL'], errors = 'coerce')
pa = pa.replace(np.nan, 0, regex = True)
#type(pa['Weighted_Average_2015_Arsenic'][0])

pa['2016 Percentage of Children with Confirmed BLLs ≥ 10 µg/dL'] = pd.to_numeric(pa['2016 Percentage of Children with Confirmed BLLs ≥ 10 µg/dL'], errors = 'coerce')
pa = pa.replace(np.nan, 0, regex = True)
#type(pa['Weighted_Average_2016_Arsenic'][0])

In [16]:
pa = pa[['County','Age-Adjusted Incidence Rate(‚Ä†) - cases per 100,000','2014 Percentage of Children with Confirmed BLLs ≥ 10 µg/dL','2015 Percentage of Children with Confirmed BLLs ≥ 10 µg/dL','2016 Percentage of Children with Confirmed BLLs ≥ 10 µg/dL','2014 Percentage of Children with Confirmed BLLs ≥ 5 µg/dL','2015 Percentage of Children with Confirmed BLLs ≥ 5 µg/dL','2016 Percentage of Children with Confirmed BLLs ≥ 5 µg/dL','Weighted_Average_2014_HAA5','Weighted_Average_2015_HAA5','Weighted_Average_2016_HAA5','Weighted_Average_2014_Arsenic','Weighted_Average_2015_Arsenic','Weighted_Average_2016_Arsenic']]


In [17]:
pa.to_csv('pennFinal2.csv')